In [1]:
import cPickle
import numpy
from keras.layers import Input
from keras.layers.embeddings import Embedding
from keras.layers.core import Dense, Dropout
from keras.layers.recurrent import LSTM
from keras.layers.merge import Concatenate
from keras.layers.wrappers import TimeDistributed
from keras.models import Model

Using TensorFlow backend.


In [4]:
train_set, valid_set, dicts = cPickle.load(open('atis.pkl', 'rb'))
word2id, tag2id = dicts['words2idx'], dicts['labels2idx']
train_x, _, train_label = train_set
val_x, _, val_label = valid_set


train_x = [[x+1 for x in w] for w in train_x]
train_label = [[x for x in w] for w in train_label]
val_x = [[x+1 for x in w] for w in val_x]
val_label = [[x for x in w] for w in val_label]
word2id = {k:v+1 for k,v in word2id.items()}

id2word  = {word2id[k]:k for k in word2id}
id2tag = {tag2id[k]:k for k in tag2id}

maxlen = 46
n_classes = len(id2tag)
n_vocab = len(id2word)

In [5]:
words_input = Input(shape=(maxlen,), dtype='int32', name='words_input')
embeddings = Embedding(input_dim=n_vocab + 1, output_dim=512, input_length=maxlen, mask_zero=True)(words_input)
embeddings = Dropout(0.5)(embeddings)
lstm_forward = LSTM(units=128, return_sequences=True, name='LSTM_forward')(embeddings)
lstm_forward = Dropout(0.5)(lstm_forward)
lstm_backward = LSTM(units=128, return_sequences=True, go_backwards=True, name='LSTM_backward')(embeddings)
lstm_backward = Dropout(0.5)(lstm_backward)
lstm_concat = Concatenate(axis=-1, name='merge_bidirections')([lstm_forward, lstm_backward])
slot_output = TimeDistributed(Dense(units=n_classes, activation='softmax'), name='slot_output')(lstm_concat)
model = Model(inputs=words_input, outputs=slot_output)
model.compile(optimizer='adam', sample_weight_mode='temporal', loss='categorical_crossentropy')

In [6]:
model.load_weights('ep=295_tagF1=0.9482frameAcc=0.8309.h5')

In [33]:
def to_vector(sentence, w2idx):
    vec = []
    for word in sentence.split():
        idx = w2idx.get(word)
        if idx == None:
            idx = w2idx['<UNK>']
        vec.append(idx)
    vec = vec[:maxlen]
    vec += [0]*(maxlen - len(vec))
    return numpy.array(vec)

In [85]:
def getNLUpred(tag_probs, tag_mask, id2tag, threshold=0.391):
    tag_txt = list()
    pred_tag_ids = numpy.argmax(tag_probs, axis=-1) * tag_mask
    for sample in pred_tag_ids:
        line_txt = [id2tag[tag_id] for tag_id in sample if tag_id != 0]
        tag_txt.append(' '.join(line_txt))
    return numpy.asarray(tag_txt)

In [145]:
sentence = "get last flight from new york to california on saturday"

In [146]:
vector = to_vector(sentence, word2id)

In [147]:
vector

array([212, 262, 194, 209, 333, 570, 503,  89, 359, 417,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0])

In [148]:
vector = vector[numpy.newaxis,:]

In [149]:
tag_probs = model.predict(vector, batch_size=1)

In [150]:
mask_array_dev = numpy.zeros_like(vector)
mask_array_dev[vector != 0] = 1

In [151]:
dev_tag_pred_txt = getNLUpred(tag_probs, mask_array_dev, id2tag)

In [152]:
list(dev_tag_pred_txt)

['O B-flight_mod O O B-fromloc.city_name I-fromloc.city_name O B-toloc.state_name O B-depart_date.day_name']